In [ ]:
import torch.nn as nn
from torch import optim

In [ ]:
text_embedding = "this is a placeholder for the text embedding that will come later"

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(len(text_embedding), 8)
        self.fc2 = nn.Linear(8, 16)
        self.fc3 = nn.Linear(16, 5) # Five OCEAN scores out
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x) # No activation function here: We want to use the sigmoid built into the loss func during training.
        return x

model = Model()
print(model)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()

In [ ]:
import torch

def train(model, train_loader, val_loader, criterion, optimizer, epochs=100):
    """
    Function to train a PyTorch model with training and validation datasets.

    Parameters:
    model: The neural network model to train.
    train_loader: DataLoader for the training dataset.
    val_loader: DataLoader for the validation dataset.
    criterion: Loss function (e.g., Binary Cross Entropy for classification).
    optimizer: Optimization algorithm (e.g., Adam, SGD).
    epochs: Number of training epochs (default=100).

    Returns:
    history: Dictionary containing loss and accuracy for both training and validation.
    """

    
    history = {'loss': [], 'val_loss': [], 'accuracy': [], 'val_accuracy': []}

    for epoch in range(epochs):
        model.train()
        total_loss, correct = 0, 0 
        
        for inputs, labels in train_loader:
            optimizer.zero_grad() 
            outputs = model(inputs).squeeze() # TODO: Is the "squeeze" needed here?
            loss = criterion(outputs, labels)
            loss.backward() 
            optimizer.step()

            total_loss += loss.item()
            correct += (torch.argmax(outputs,dim=1) == labels).sum().item()

        train_loss = total_loss / len(train_loader)
        train_acc = correct / len(train_loader.dataset)

        model.eval()
        val_loss, val_correct = 0, 0
        with torch.no_grad(): 
            for inputs, labels in val_loader:
                outputs = model(inputs).squeeze() # TODO: Is the "squeeze" needed here?
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (torch.argmax(outputs,dim=1) == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['accuracy'].append(train_acc)
        history['val_accuracy'].append(val_acc)

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    return history  # Return training history

# Train the model
history = train(model, train_loader, val_loader, criterion, optimizer, epochs=50)

In [ ]:
# Our criterion nn.BCEWithLogitsLoss() has a sigmoid function built in, so we left out the sigmoid in the model.
# This means we need to manually run the output through a sigmoid function when making predictions on a "finished" model.

user_input = "this is a placeholder for the data input by the user when the model is deployed"

with torch.no_grad():
    logits = model(user_input)
    probs = torch.sigmoid(logits)
    predictions = (probs > 0.5).int()